In [1]:
import pandas as pd
import os
import numpy as np 

In [24]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [25]:
from sklearn.metrics.pairwise import cosine_similarity

In [44]:
import chardet

In [62]:
nat_data_path = 'data/raw/national_election/eu_ned_national_nuts2(1).csv'

In [66]:
df = pd.read_csv(nat_data_path,delimiter=",", encoding='UTF-8')

In [14]:
lissy = pd.read_csv('data/cleaned/lissy/france_gini.csv')

In [67]:
france = df[df['country'] == 'France']

In [19]:
joined = france.merge(lissy, left_on= ['regionname','year'], right_on= ['region','year'], how='left')

the above join has far too high a miss rate, going to use the ml approach from the regional and see if it worked. 

In [69]:
lissy['cleaned_region'] = lissy['region'].str.lower().str.replace('[^\w\s]', '')
france['cleaned_region'] = france['regionname'].str.lower().str.replace('[^\w\s]', '')

C:\Users\samtg\AppData\Local\Temp\ipykernel_8708\3880206945.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france['cleaned_region'] = france['regionname'].str.lower().str.replace('[^\w\s]', '')


In [27]:
tfidf_vectorizer = TfidfVectorizer()

In [80]:
X_lissy = tfidf_vectorizer.fit_transform(lissy['cleaned_region'])
X_france = tfidf_vectorizer.transform(france['cleaned_region'])

In [81]:
similarity_matrix = cosine_similarity(X_france, X_lissy)

In [82]:
similarity_threshold = 0.7

In [83]:
predicted_matches = []
confidence_scores = []

In [84]:
for i in range(len(france)):
    best_match_index = np.argmax(similarity_matrix[i])
    best_similarity_score = similarity_matrix[i][best_match_index]
    
    if best_similarity_score > similarity_threshold:
        predicted_match = lissy.iloc[best_match_index]['region']
    else:
        predicted_match = None
    
    predicted_matches.append(predicted_match)
    confidence_scores.append(best_similarity_score)

In [85]:
france['predicted_region'] = predicted_matches
france['confidence_score'] = confidence_scores

C:\Users\samtg\AppData\Local\Temp\ipykernel_8708\2462273079.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france['predicted_region'] = predicted_matches
C:\Users\samtg\AppData\Local\Temp\ipykernel_8708\2462273079.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france['confidence_score'] = confidence_scores


In [94]:
france

,country,country_code,nutslevel,nuts2,regionname,type,year,party_abbreviation,party_english,party_native,partyfacts_id,partyvote,electorate,totalvote,validvote,cleaned_region,predicted_region,confidence_score
3354,France,FR,2,FR10,Ile-de-France,Parliament,1993,DIV,other,other,1898.0,124779.0,6059295.0,4095568.0,3945280.0,ile-de-france,Ile-de-France,1.0
3355,France,FR,2,FR10,Ile-de-France,Parliament,1993,DIVED,NaN,NaN,452.0,36001.0,6059295.0,4095568.0,3945280.0,ile-de-france,Ile-de-France,1.0
3356,France,FR,2,FR10,Ile-de-France,Parliament,1993,FN,National Front / National Rally,Front National / Rassemblement national,433.0,561509.0,6059295.0,4095568.0,3945280.0,ile-de-france,Ile-de-France,1.0
3357,France,FR,2,FR10,Ile-de-France,Parliament,1993,GE++,Ecology Generation,Generation ecologie,1273.0,199932.0,6059295.0,4095568.0,3945280.0,ile-de-france,Ile-de-France,1.0
3358,France,FR,2,FR10,Ile-de-France,Parliament,1993,MAJ,Party of Presidential Majority,Parti de la Majorite presidentielle,3157.0,28761.0,6059295.0,4095568.0,3945280.0,ile-de-france,Ile-de-France,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5566,France,FR,2,FRY5,Mayotte,Parliament,2017,OTHER LEFT,other left,divers gauche,126.0,4234.0,82802.0,36721.0,32914.0,mayotte,None,0.0
5567,France,FR,2,FRY5,Mayotte,Parliament,2017,OTHER RIGHT,other right,divers droite,1776.0,2876.0,82802.0,36721.0,32914.0,mayotte,None,0.0
5568,France,FR,2,FRY5,Mayotte,Parliament,2017,PRG,Radical Party of the Left,Parti radical de gauche,1783.0,990.0,82802.0,36721.0,32914.0,mayotte,None,0.0
5569,France,FR,2,FRY5,Mayotte,Parliament,2017,PS,Socialist Party,Parti socialiste,1478.0,4520.0,82802.0,36721.0,32914.0,mayotte,None,0.0


In [ ]:
france_joined = france.merge(lissy, left_on= ['predicted_region','year'], right_on= ['region','year'], how='left')

In [97]:
france_joined['type'].value_counts()

type
Parliament    2217
Name: count, dtype: int64

In [101]:
lissy.columns

Index(['region', 'year', 'avg_gini', 'cleaned_region'], dtype='object')

In [104]:
election_years = [1993, 1997, 2002, 2007, 2012, 2017]

In [109]:
result_df = pd.DataFrame(columns=['election_year', 'year','region', 'avg_gini', 'delta_gini'])

In [111]:
result_df

,election_year,year,region,avg_gini,delta_gini


In [103]:
lissy = lissy.sort_values(by=['year'])

In [140]:
result_list = []

In [141]:
for year in lissy['year'].unique():
    # Iterate through all regions
    for region in lissy['region'].unique():
        # Filter rows in 'lissy' for the current year and region
        year_region_data = lissy[(lissy['year'] == year) & (lissy['region'] == region)]
        
        # Check if there is data for the current year and region
        if not year_region_data.empty:
            # Calculate the average Gini for the current year and region
            avg_gini = year_region_data['avg_gini'].mean()
            
            # Calculate the change in Gini since the last year within the same region
            prev_year_data = lissy[(lissy['year'] == year - 1) & (lissy['region'] == region)]
            
            if not prev_year_data.empty:
                prev_year_gini = prev_year_data['avg_gini'].iloc[0]
                change_in_gini = avg_gini - prev_year_gini
            else:
                change_in_gini = None
            
            # Create a DataFrame for the current year and region
            result = pd.DataFrame({'year': year,
                                   'region': region,
                                   'avg_gini': avg_gini,
                                   'change_gini': change_in_gini}, index=[0])
            
            # Append the DataFrame to the list
            result_list.append(result)

# Concatenate all DataFrames in the list into a single DataFrame
result_df = pd.concat(result_list, ignore_index=True)


In [142]:
result_df

,year,region,avg_gini,change_gini
0,1996,Bourgogne,0.274,None
1,1996,Lorraine,0.291,None
2,1996,Nord-Pas de Cal,0.274,None
3,1996,Limousin,0.290,None
4,1996,Languedoc-Rouss,0.319,None
...,...,...,...,...
494,2018,Rhône-Alpes,0.316,0.008
495,2018,Champagne-Arden,0.301,0.002
496,2018,Alsace,0.314,0.024
497,2018,Corse,0.312,0.027


In [146]:
france.dropna(subset=['predicted_region'], inplace=True)

C:\Users\samtg\AppData\Local\Temp\ipykernel_8708\783455990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  france.dropna(subset=['predicted_region'], inplace=True)


In [148]:
result_df['year'].sort_values()

0      1996
19     1996
18     1996
17     1996
16     1996
       ... 
478    2018
477    2018
497    2018
486    2018
498    2018
Name: year, Length: 499, dtype: int64

I am going to snip every value before '96 because that's all I have from LISSY right now, may be possible to go back and get more

In [149]:
france = france[france['year'] >= 1996] 

now we join 

In [150]:
france_joined = france.merge(result_df, left_on= ['predicted_region','year'], right_on= ['region','year'], how='left')

In [201]:
france_joined

,country,country_code,nutslevel,nuts2,regionname,type,year,party_abbreviation,party_english,party_native,...,partyvote,electorate,totalvote,validvote,cleaned_region,predicted_region,confidence_score,region,avg_gini,change_gini
0,France,FR,2,FR10,Ile-de-France,Parliament,1997,DIV,other,other,...,39049.0,6070933.0,3959761.0,3821448.0,ile-de-france,Ile-de-France,1.0,Ile-de-France,0.308,0.011
1,France,FR,2,FR10,Ile-de-France,Parliament,1997,DIVED,NaN,NaN,...,4354.0,6070933.0,3959761.0,3821448.0,ile-de-france,Ile-de-France,1.0,Ile-de-France,0.308,0.011
2,France,FR,2,FR10,Ile-de-France,Parliament,1997,FN,National Front / National Rally,Front National / Rassemblement national,...,587557.0,6070933.0,3959761.0,3821448.0,ile-de-france,Ile-de-France,1.0,Ile-de-France,0.308,0.011
3,France,FR,2,FR10,Ile-de-France,Parliament,1997,OTHER,NaN,NaN,...,6686.0,6070933.0,3959761.0,3821448.0,ile-de-france,Ile-de-France,1.0,Ile-de-France,0.308,0.011
4,France,FR,2,FR10,Ile-de-France,Parliament,1997,OTHER FAR-LEFT,other far-left,extreme gauche,...,139786.0,6070933.0,3959761.0,3821448.0,ile-de-france,Ile-de-France,1.0,Ile-de-France,0.308,0.011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,France,FR,2,FRM0,Corse,Parliament,2017,OTHER LEFT,other left,divers gauche,...,2710.0,233532.0,115221.0,112494.0,corse,Corse,1.0,Corse,0.285,0.01
1652,France,FR,2,FRM0,Corse,Parliament,2017,OTHER RIGHT,other right,divers droite,...,4700.0,233532.0,115221.0,112494.0,corse,Corse,1.0,Corse,0.285,0.01
1653,France,FR,2,FRM0,Corse,Parliament,2017,PCF,French Communist Party,Parti communiste francaise,...,4439.0,233532.0,115221.0,112494.0,corse,Corse,1.0,Corse,0.285,0.01
1654,France,FR,2,FRM0,Corse,Parliament,2017,REG,NaN,NaN,...,33925.0,233532.0,115221.0,112494.0,corse,Corse,1.0,Corse,0.285,0.01


ok let's try parties 

In [171]:
crosswalk = pd.read_csv('data/raw/crosswalks/manif_cross.csv')

In [169]:
manifesto_full = pd.read_csv('data/raw/parties/manifesto_full.csv')

C:\Users\samtg\AppData\Local\Temp\ipykernel_8708\2885752569.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  manifesto_full = pd.read_csv('data/raw/parties/manifesto_full.csv')


In [174]:
crosswalk_fr = crosswalk[crosswalk['country'] == 'FRA']

In [178]:
france_joined.columns

Index(['country', 'country_code', 'nutslevel', 'nuts2', 'regionname', 'type',
       'year', 'party_abbreviation', 'party_english', 'party_native',
       'partyfacts_id', 'partyvote', 'electorate', 'totalvote', 'validvote',
       'cleaned_region', 'predicted_region', 'confidence_score', 'region',
       'avg_gini', 'change_gini'],
      dtype='object')

In [180]:
crosswalk_fr.columns

Index(['country', 'dataset_key', 'dataset_party_id', 'name_short', 'name',
       'name_english', 'year_first', 'year_last', 'share', 'share_year',
       'description', 'comment', 'created', 'modified', 'external_id',
       'partyfacts_id', 'linked'],
      dtype='object')

In [181]:
france_with_parties = france_joined.merge(crosswalk_fr, on='partyfacts_id', how='left')

In [184]:
#can try to find a bigger crosswalk later but for now just want parties I can know about 
france_with_parties.dropna(subset=['partyfacts_id'], inplace=True)

In [212]:
manifesto_france_only = manifesto_full[manifesto_full['countryname'] == 'France']

In [210]:
manifesto_full['year'] = manifesto_full['edate'].str.split('/').str[-1].astype(int)

In [213]:
france_manifesto = france_with_parties.merge(manifesto_france_only, left_on= ['dataset_party_id','year'], right_on= ['party','year'] ,how='left')

In [214]:
france_manifesto.dropna(subset=['partyname'], inplace=True)

In [215]:
france_manifesto[['regionname','year','party_english','partyname','partyfacts_id','party','dataset_party_id']]

,regionname,year,party_english,partyname,partyfacts_id,party,dataset_party_id
2,Ile-de-France,1997,National Front / National Rally,National Front,433.0,31720.0,31720.0
6,Ile-de-France,1997,French Communist Party,French Communist Party,1251.0,31220.0,31220.0
8,Ile-de-France,1997,Socialist Party,Socialist Party,1478.0,31320.0,31320.0
9,Ile-de-France,1997,Union for French Democracy,Union for French Democracy,213.0,31624.0,31624.0
10,Ile-de-France,1997,Greens,The Greens,1108.0,31110.0,31110.0
...,...,...,...,...,...,...,...
1458,Corse,2012,NaN,Union for a Popular Movement,1595.0,31626.0,31626.0
1462,Corse,2017,Unbowed France,Indomitable France,5858.0,31240.0,31240.0
1463,Corse,2017,National Front / National Rally,National Front,433.0,31720.0,31720.0
1467,Corse,2017,French Communist Party,French Communist Party,1251.0,31220.0,31220.0


In [216]:
france_manifesto.to_csv('data/cleaned/national/france_manifesto.csv')